## Image classification of MNISTfashion dataset with importing ResNet pre-trained model

### Importing Modules

In [105]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader
import tkinter as tk
from PIL import Image, ImageTk

In [107]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Transformation for FashionMNIST dataset

In [110]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  
    transforms.Resize((224, 224)),                
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


### Loading the dataset

In [113]:
train_dataset = datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)

# Limitung the size of dataset for faster processing (subset of 1000 for training and 500 for testing)
train_dataset.data = train_dataset.data[:1000]  
train_dataset.targets = train_dataset.targets[:1000]

test_dataset.data = test_dataset.data[:500]  
test_dataset.targets = test_dataset.targets[:500]

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

### Loading the pre-trained Model

In [116]:
model = models.resnet18(pretrained=True)

### Freezing all layers (for feature extraction)

In [119]:
for param in model.parameters():
    param.requires_grad = False

#### Removing final fully connected layer

In [122]:
model = nn.Sequential(*list(model.children())[:-1])  
model = model.to(device)


#### Function to extract features

In [125]:
def extract_features(loader):
    model.eval()  
    features = []
    labels = []
    
    with torch.no_grad():  # Disable gradient computation
        for images, targets in loader:
            images = images.to(device)
            outputs = model(images)  
            outputs = outputs.view(outputs.size(0), -1)  
            features.append(outputs.cpu())  
            labels.append(targets)
    
    # Concatenate all batches
    features = torch.cat(features)
    labels = torch.cat(labels)
    
    return features, labels

#### Extracting features from the training and test sets

In [128]:
train_features, train_labels = extract_features(train_loader)
test_features, test_labels = extract_features(test_loader)


#### Simple classifier to train on new dataset

In [130]:
classifier = nn.Linear(train_features.size(1), 10)  
classifier = classifier.to(device)

### Defining optimizer and loss function

In [134]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=0.001)

### Training the classifier on extracted features

In [137]:
def calculate_accuracy(outputs, labels):
    _, predicted = torch.max(outputs, 1)  
    correct = (predicted == labels).sum().item() 
    return correct

num_epochs = 5
for epoch in range(num_epochs):
    classifier.train() 
    running_loss = 0.0
    total_correct = 0
    total_samples = 0
    
    indices = torch.randperm(train_features.size(0))
    train_features = train_features[indices]
    train_labels = train_labels[indices]
    
    for i in range(0, len(train_features), 64):
        batch_features = train_features[i:i+64].to(device)
        batch_labels = train_labels[i:i+64].to(device)
        
        outputs = classifier(batch_features)
        loss = criterion(outputs, batch_labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        total_correct += calculate_accuracy(outputs, batch_labels)
        total_samples += batch_labels.size(0)

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = total_correct / total_samples * 100  
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

Epoch [1/5], Loss: 1.9976, Accuracy: 37.70%
Epoch [2/5], Loss: 1.2048, Accuracy: 74.40%
Epoch [3/5], Loss: 0.8545, Accuracy: 80.10%
Epoch [4/5], Loss: 0.6981, Accuracy: 82.40%
Epoch [5/5], Loss: 0.6109, Accuracy: 83.20%


In [139]:
def calculate_accuracy(classifier, features, labels):
    classifier.eval()  # Set classifier to evaluation mode
    correct = 0
    total = 0
    
    with torch.no_grad():  # Disable gradient calculation for efficiency
        outputs = classifier(features.to(device))  # Make predictions
        _, predicted = torch.max(outputs, 1)  # Get the predicted class
        correct = (predicted == labels.to(device)).sum().item()  # Compare with actual labels
        total = labels.size(0)  # Get the total number of samples
    
    accuracy = 100 * correct / total
    return accuracy

# Calculate accuracy on the test set
test_accuracy = calculate_accuracy(classifier, test_features, test_labels)
print(f"Test Accuracy: {test_accuracy:.2f}%")

Test Accuracy: 81.20%


### Tkinter window for displaying images

In [142]:
window = tk.Tk()
window.title("FashionMNIST Prediction")

image_label = tk.Label(window)
image_label.pack()
prediction_label = tk.Label(window, text="")
prediction_label.pack()

test_images_raw = []

for images, labels in test_loader:
    test_images_raw.append((images, labels))
    break 

image_index = 0

### Function to make predictions and update GUI

In [145]:
def update_prediction(image_index):
    classifier.eval()
    

    image = test_images_raw[0][0][image_index]  
    feature = test_features[image_index].unsqueeze(0).to(device)  
    
    
    with torch.no_grad():
        output = classifier(feature)
        _, predicted_label = torch.max(output, 1)
    
    # Map FashionMNIST label indices to human-readable names
    label_map = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
                 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
    predicted_name = label_map[predicted_label.item()]
    
    # Convert the image to PIL format for Tkinter display
    image = image.cpu().numpy().transpose(1, 2, 0)  # Convert from Tensor to numpy
    image = (image * 0.5 + 0.5) * 255  
    image = Image.fromarray(image.astype('uint8'))

    
    img_tk = ImageTk.PhotoImage(image)
    image_label.config(image=img_tk)
    image_label.image = img_tk 
    prediction_label.config(text=f"Predicted: {predicted_name}")


def next_image():
    global image_index
    image_index = (image_index + 1) % len(test_images_raw[0][0])  
    update_prediction(image_index)


next_button = tk.Button(window, text="Next Image", command=next_image)
next_button.pack()


update_prediction(image_index)


window.mainloop()